In [1]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
import shutil
import cv2
TRAINING_DIR = "Train"

training_datagen = ImageDataGenerator(rescale = 1./255,
                                      horizontal_flip=True,
																			rotation_range=30,
																			height_shift_range=0.2,
                                      fill_mode='nearest')

VALIDATION_DIR = "Test"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(256,256),
	class_mode='categorical',
  batch_size = 32
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(256,256),
	class_mode='categorical',
  batch_size= 32
)

Found 8999 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


In [2]:
from tensorflow.keras.optimizers import RMSprop,Adam
model = tf.keras.models.Sequential([
          tf.keras.layers.Conv2D(96, (11,11), strides=(4,4), activation='relu', input_shape=(256, 256, 3)),
          tf.keras.layers.MaxPooling2D(pool_size = (3,3), strides=(2,2)),
          tf.keras.layers.Conv2D(256, (5,5), activation='relu'),
          tf.keras.layers.MaxPooling2D(pool_size = (3,3), strides=(2,2)),
          tf.keras.layers.Conv2D(384, (5,5), activation='relu'),
          tf.keras.layers.MaxPooling2D(pool_size = (3,3), strides=(2,2)),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Dense(2048, activation='relu'),
          tf.keras.layers.Dropout(0.25),
          tf.keras.layers.Dense(1024, activation='relu'),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Dense(8, activation='softmax')
])
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001),
              metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 30, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 256)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 384)         2457984   
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 3, 3, 384)        0

c:\Users\Anirudh\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [3]:
# class myCallback(tf.keras.callbacks.Callback):
#   def on_epoch_end(self, epoch, logs={}):
#     if(logs.get('val_acc')>=0.98):
#       print('\nReached ^98%')
#       self.model.stop_training = True
# callbacks = myCallback()

history = model.fit(
    train_generator,
    steps_per_epoch = int(4000/32),
    epochs = 20,
    validation_data = validation_generator,
    validation_steps = (1000)
    #callbacks=[callbacks]
)

Epoch 1/20
  4/125 [..............................] - ETA: 3:53 - loss: 2.0526 - acc: 0.1719

In [ ]:
model.save('my_model.h5')
new_model = tf.keras.models.load_model('my_model.h5')

In [ ]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# acc = history.history['acc']
# val_acc = history.history['val_acc']
# loss = history.history['loss']
# val_loss = history.history['val_loss']

# epochs = range(len(acc))

# plt.plot(epochs, acc, 'g', label='Training accuracy')
# plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
# plt.title('Training and validation accuracy')
# plt.legend(loc=0)
# plt.figure()
# plt.show()

# plt.plot(epochs, loss, 'r', label='Training loss')
# plt.plot(epochs, val_loss, 'orange', label='Validation loss')
# plt.title('Training and validation loss')

# plt.legend(loc=0)
# plt.figure()
# plt.show()

In [ ]:
for fn in uploaded.keys():
  path = '/content/' + fn
  img = image.load_img(path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0) /255
  print(np.argmax(classes[0])==0, max(classes[0]))

In [ ]:

import numpy as np
cap=cv2.VideoCapture(0)

ofile=open("out.txt","w")
while True:
      
    _,frame=cap.read()
    

    cv2.imwrite("capture.jpg",frame)
    

    
    img = image.load_img("capture.jpg", target_size=(224, 224))
    
    
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0) /255
    classes = model.predict(x)

    if max(classes[0])*100>=80:
      print("Fire!!!!!!!")
      img=cv2.putText(frame,str(classes),(10,230),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2,cv2.LINE_AA)
      cv2.imwrite("output.jpg",img)
      
    else:
      print("NO fire!!!!!!!!!!!")
      img=cv2.putText(frame,str(classes),(10,230),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2,cv2.LINE_AA)
      cv2.imwrite("output.jpg",img)


In [ ]:
import cv2
import numpy as np
from tensorflow import keras
# from keras import load_model


def model_predict(img_path, model):
    
    #update by ViPS
    img = cv2.imread(img_path)
    print(img.shape)
    new_arr = np.reshape(img,(224,224,3))
    # new_arr = np.
    new_arr = np.array(new_arr/255)
    new_arr = new_arr.reshape(1,256,256,3)
    print(new_arr.shape)
    
    preds = model.predict(new_arr)
    return preds

model_predict('Test\Apple_Scab\image (902).JPG',keras.models.load_model("my_model.h5"))